<a href="https://colab.research.google.com/github/jerry71867/AI-/blob/main/17_Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi
#在 NVIDIA GPU 系統上查詢 GPU 相關的信息。

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
!pip install transformers==4.29.0 -U
#安裝 transformers 的 Python 套件。
!pip install nlp -U
#安裝 nlp 的 Python 套件。
!pip install torch==2.0.0 -U
#安裝 torch 的 Python 套件。
!pip install transformers accelerate
"""
安裝 transformers 和 accelerate 這兩個 Python 套件。
transformers用於自然語言處理的庫，用於訓練和部署深度學習模型。
accelerate用於加速深度學習訓練的庫，可以有效地利用多個 GPU 或分布式訓練。
"""

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 29.0 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.2
    Uninstalling transformers-4.38.2:
      Successfully uninstalled transformers-4.38.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 42.9 MB/s eta 0:00:00
  

'\n安裝 transformers 和 accelerate 這兩個 Python 套件。\ntransformers用於自然語言處理的庫，用於訓練和部署深度學習模型。\naccelerate用於加速深度學習訓練的庫，可以有效地利用多個 GPU 或分布式訓練。\n'

In [ ]:
from transformers import BertForSequenceClassification, BertTokenizerFast, Trainer, TrainingArguments
"""
從 transformers 模塊中導入了四個類：BertForSequenceClassification、BertTokenizerFast、Trainer 和 TrainingArguments。
BertForSequenceClassification用於文本分類任務的 BERT 模型。
BertTokenizerFast用於快速分詞的 BERT 分詞器。
Trainer用於訓練深度學習模型的類。
TrainingArgument包含訓練相關參數的類。
"""
from nlp import load_dataset, Dataset
"""
從 nlp 模塊中導入了兩個函數
load_dataset用於加載數據集的函數，通常用於自然語言處理任務。
Dataset用於表示數據集
"""
import nlp
#導入 nlp 模塊。
import torch
#導入 torch 模塊，這是 PyTorch 深度學習框架的主要模塊。
import random
#導入 Python 標準庫中的 random 模塊，用於生成隨機數。
import pandas as pd
#導入 pandas 模塊，用於數據分析和操作的流行庫。
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
"""
從 sklearn.metrics 模塊中導入了四個函數：accuracy_score、precision_recall_fscore_support。
accuracy_score計算分類準確度的函數。
precision_recall_fscore_support計算分類的準確率、召回率、F1 分數和支持度的函數。
"""

'\n從 sklearn.metrics 模塊中導入了四個函數：accuracy_score、precision_recall_fscore_support。\naccuracy_score計算分類準確度的函數。\nprecision_recall_fscore_support計算分類的準確率、召回率、F1 分數和支持度的函數。\n'

In [ ]:
!wget https://github.com/shhuangmust/AI/raw/112-1/train.csv
#用於下載文件的命令

--2024-04-15 09:37:47--  https://github.com/shhuangmust/AI/raw/112-1/train.csv
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/shhuangmust/AI/112-1/train.csv [following]
--2024-04-15 09:37:47--  https://raw.githubusercontent.com/shhuangmust/AI/112-1/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 388895 (380K) [text/plain]
Saving to: ‘train.csv’

train.csv           100%[===================>] 379.78K  --.-KB/s    in 0.03s   

2024-04-15 09:37:47 (10.8 MB/s) - ‘train.csv’ saved [388895/388895]



In [ ]:
df = pd.read_csv('train.csv')
#使用pandas的read_csv函式從名為'train.csv'的CSV檔案中讀取資料並將其儲存在一個pandas的DataFrame中。
dataset = Dataset.from_pandas(df)
#將 pandas 的 DataFrame 轉換為 nlp 庫中的資料集 Dataset。


In [ ]:
df.sample(10)
#從DataFrame中隨機抽取指定數量的樣本並顯示。

,ID,Name,Description,label
464,464,頂級月子餐,「生化湯幫助排除惡露...發汝茶藥膳湯讓產後泌乳順暢...雞湯加強水分代謝...杜仲茶緩解腰...,1
649,649,愛尚它?每日精選組合隨身包,愛尚它?產品係經過科學研擬，以果糖、D-葡萄糖、檸檬酸、碳酸氫鉀和其它成分精心配製而成，當您...,0
117,117,老薑桂圓紅棗茶磚,「還能促進膽汁合成速率...(紅茶)...有利尿功能...(黑糖)止疼、行血、活血散寒......,1
156,156,宏醫百大蔬果素食B群補給組,「55-64歲30天-5公斤、65-74歲30天-2公斤...斯容：『那我吃B群可以治療代謝...,1
452,452,紅●原萃,「人體的免疫力，抵抗自由基對細胞的破壞，減少癌症的發生，特別是男性的攝護腺癌，同時也是天然的...,1
44,44,樂視C,「樂視C」「藍光專利…明亮好視野…經由美國國家眼科研究所大型臨床實驗結果…可提供全家人的優視...,1
149,149,Nu-Lax柳丁味排毒粉300g&排毒顆粒100粒,「Nu-Lax柳丁味排毒粉300g&排毒顆粒100粒」「1.幫助自然排毒…3.預防癌症…5....,1
599,599,【GNC健安喜】三效魚油1500膠囊60顆(一顆抵3顆),每顆含有EPA及DHA，含量為一般魚油的三倍。-採最新純化技術濃縮萃取，純度高，符合汞、鎘、...,0
207,207,『禮盒綜合款』極品銀耳燕窩飲膠原蛋白/珍珠粉【兩種口味各三瓶】贈送精美提袋,「多?類物質-表皮生長因子，簡生EGF被譽為『美容基因』...美肌保濕...很受減重族群的青...,1
261,261,醫之方SOD養生本草酵素組,「長庚醫院亞洲第一型抗癌藥物...長庚醫院亞洲最大質子放射治療...抵抗力下降、慢性發炎、過...,1


In [ ]:
model = BertForSequenceClassification.from_pretrained('hfl/chinese-bert-wwm-ext')
"""
載入預先訓練好的 BERT 模型。
BertForSequenceClassification用於文本分類任務的 BERT 模型。
from_pretrained() 是 transformers 模組中的一個方法，用於從預訓練的模型中載入已訓練好的參數。
'hfl/chinese-bert-wwm-ext' 是要載入的預訓練模型的識別符或路徑。
"""

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of the model checkpoint at hfl/chinese-bert-wwm-ext were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkp

"\n載入預先訓練好的 BERT 模型。\nBertForSequenceClassification用於文本分類任務的 BERT 模型。\nfrom_pretrained() 是 transformers 模組中的一個方法，用於從預訓練的模型中載入已訓練好的參數。\n'hfl/chinese-bert-wwm-ext' 是要載入的預訓練模型的識別符或路徑。\n"

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained('hfl/chinese-bert-wwm-ext')
"""
載入預先訓練好的 BERT 分詞器。
BertTokenizerFast用於快速分詞的 BERT 分詞器。
from_pretrained() 是 transformers 模組中的一個方法，用於從預訓練的模型中載入分詞器。
'hfl/chinese-bert-wwm-ext' 是要載入的預訓練模型的識別符或路徑。
"""

tokenizer_config.json:   0%|          | 0.00/19.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

"\n載入預先訓練好的 BERT 分詞器。\nBertTokenizerFast用於快速分詞的 BERT 分詞器。\nfrom_pretrained() 是 transformers 模組中的一個方法，用於從預訓練的模型中載入分詞器。\n'hfl/chinese-bert-wwm-ext' 是要載入的預訓練模型的識別符或路徑。\n"

In [ ]:
RANDOM_SEED = 5
#用於設置隨機數生成器的種子，以確保在相同的種子下，生成的隨機數序列是相同的，從而實現可重現的結果
MAX_LEN = 512
#用於設置文本序列的最大長度。
EPOCHS=5
#用於設置訓練模型時的迭代次數，每個epoch表示對整個訓練數據集的一次完整遍歷。
BATCH_SIZE = 8
#用於設置訓練過程中每個小批次的樣本數量。

In [ ]:
shuffled_ds = dataset.shuffle(RANDOM_SEED)
"""
將dataset進行洗牌操作。
shuffle() 是 Dataset 物件的方法，用於隨機打亂資料集中的樣本。
RANDOM_SEED先前定義的隨機種子，用於確保在不同的執行中得到相同的隨機排序。
"""
split_ds = shuffled_ds.train_test_split(test_size=0.2)
"""
將洗牌後的資料集拆分為訓練集和測試集。
train_test_split() 是 Dataset 物件的方法，用於將資料集分割成訓練集和測試集。
test_size=0.2表示將 20% 的樣本分配給測試集。
"""
train_dataset = split_ds['train']
#將拆分後的訓練集存儲在變數 train_dataset 中。
test_val_dataset = split_ds['test']
#將拆分後的測試集存儲在變數 test_val_dataset 中。
split_tv = test_val_dataset.train_test_split(test_size=0.5)
"""
測試集進一步拆分為測試集和驗證集。
train_test_split() 方法用於將資料集分割成訓練集和測試集。
test_size=0.5 表示將 50% 的樣本分配給驗證集。
"""
test_dataset = split_tv['train']
#拆分後的測試集存儲在變數 test_dataset 中。
val_dataset = split_tv['test']
#拆分後的驗證集存儲在變數 val_dataset 中。

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
train_dataset[2]
#用於從訓練資料集中獲取第三個樣本

{'Description': '「(六六大順)..氧化..都是較天然..他這個就是好大便...這個是天然的肚子不會痛金放心..如果你不通那樣積住了便祕...(珍珠精)日本的珍珠精..不論嬰兒，嬰兒發育會變很快..養不大那種..如果是有孕的婦女..因為母體需要很多營養..鈣質常常流失...出生的嬰兒吃身體勇健...吃2-3月之後你可以看你的指甲還有牙齒更加勇...男生的話最明顯就看鬍鬚...如果老人家有斑點的，皮膚阿也會更水...(咖啡)跟大家介紹醫學健康雜誌研究出來..喝咖啡可以減少心臟病跟心肌梗塞這個大家都知道..還可以預防老人癡呆症這種重大疾病，現在很多人在研究..老人失智症這是很多人都很怕…降低6成以上阿茲海默症跟老人癡呆症的風險，喝咖啡可以增加我們身體一種成分GCSFA，咖啡可以保護我們腦部的蛋白，咖啡一個好處是可以顧肝...喝咖啡可以穩定血糖可以讓人更長壽」',
 'ID': 51,
 'Name': '六六大順、珍珠精、咖啡',
 'label': 1}

In [ ]:
tokenizer.tokenize("體重跟體脂都有在持續緩慢的下降中")
"""
將一個中文句子進行分詞。
tokenizer 是之前創建的 BERT 分詞器物件。
tokenize() 是分詞器物件的一個方法，用於將輸入的句子分詞。
"""

['體',
 '重',
 '跟',
 '體',
 '脂',
 '都',
 '有',
 '在',
 '持',
 '續',
 '緩',
 '慢',
 '的',
 '下',
 '降',
 '中']

In [ ]:
def tokenize(batch):
#定義 tokenize 的函式，用於對資料集中的批次進行分詞處理。
  return tokenizer(batch['Description'], max_length=MAX_LEN, padding=True, truncation=True)
  """
  在 tokenize 函式中，使用BERT分詞器tokenize對輸入的批次中的Description欄位進行分詞處理。
  max_length=MAX_LEN設置最大序列長度為之前定義的MAX_LEN。
  padding=True對輸出的序列進行填充，使得所有序列長度一致。
  truncation=True對輸入的序列進行截斷，使得所有序列的長度不超過指定的最大長度。
  """
train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
#使用map()方法將tokenize函式應用到訓練資料集train_dataset的每個批次。
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))
#對測試資料集進行相同的處理。
val_dataset = val_dataset.map(tokenize, batched=True, batch_size=len(val_dataset))
#對驗證資料集進行相同的處理。

train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
#將訓練資料集的格式設置為PyTorch張量格式，並指定需要的欄位為input_ids、attention_mask和label。
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
#將測試資料集的格式設置為PyTorch張量格式，並指定需要的欄位為input_ids、attention_mask和label。
val_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
#將驗證資料集的格式設置為PyTorch張量格式，並指定需要的欄位為input_ids、attention_mask和label。

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
def compute_metrics(pred):
#定義 compute_metrics 的函式，用於計算模型訓練過程中的指標。
    labels = pred.label_ids
    #從模型預測結果 pred 中獲取真實的標籤。
    preds = pred.predictions.argmax(-1)
    #從模型預測結果 pred 中獲取預測的類別，取最大值的索引作為預測結果。
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    """
    使用 precision_recall_fscore_support 函式計算精確率、召回率和 F1 分數。
    average='binary' 表示計算二元分類的平均值。
    """
    acc = accuracy_score(labels, preds)
    #使用 accuracy_score 函式計算準確率。
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

training_args = transformers.TrainingArguments(
    output_dir='./results',
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    warmup_steps=50,
    weight_decay=0.01,
    # evaluate_during_training=True,
    logging_dir='./logs',
)
#定義了模型訓練的參數

trainer = transformers.Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)
"""
創建了一個 Trainer 對象，用於執行模型的訓練和評估。
model：訓練的模型。
args：訓練的相關參數。
compute_metrics：自定義的計算指標函式。
train_dataset：訓練資料集。
eval_dataset：評估資料集。
"""

NameError: name 'transformers' is not defined

In [ ]:
trainer.train()

trainer.evaluate()


NameError: name 'trainer' is not defined

In [ ]:
print(df.iloc[220]['label'], df.iloc[220]['Description'])
print(df.iloc[578]['label'], df.iloc[578]['Description'])



1 「增強免疫力...護眼明目...抗氧化...保肝...皮膚：...保護皮膚，減緩皮屑問題，降低敏感...毛髮……促進毛髮亮麗...消化系統」「強化免疫系統，有助調整過敏體質...清除自由基延緩老化...調整腸胃道機能...調理皮膚問題...諾麗果含有許多強大的抗氧化劑」「阻止或減少病痛的發生，並迅速修補已受傷的細胞，將代謝殘渣或毒素排出體外」「它能維護身體細胞組織正常運作，阻止或減少病痛的發生，並迅速修補已受傷的細胞，將代謝殘渣或毒素排出體外，使身體恢復正常」
0 ·???????每份含有30mg輔Q10。輔Q10可參與維生素E再生，兩者具協同作用，可促進新陳代謝。保護循環順暢維持健康。暢銷日本的最佳美容聖品。


In [ ]:
#test_text = '增強免疫力...護眼明目...抗氧化...保肝...皮膚：...保護皮膚，減緩皮屑問題，降低敏感...毛髮……促進毛髮亮麗...消化系統」「強化免疫系統，有助調整過敏體質...清除自由基延緩老化...調整腸胃道機能...調理皮膚問題...諾麗果含有許多強大的抗氧化劑」「阻止或減少病痛的發生，並迅速修補已受傷的細胞，將代謝殘渣或毒素排出體外」「它能維護身體細胞組織正常運作，阻止或減少病痛的發生，並迅速修補已受傷的細胞，將代謝殘渣或毒素排出體外，使身體恢復正常'
test_text = '???????每份含有30mg輔Q10。輔Q10可參與維生素E再生，兩者具協同作用，可促進新陳代謝。保護循環順暢維持健康。暢銷日本的最佳美容聖品'
#定義一個 test_text 的變數，其中包含待處理的文本序列。
MAX_LEN = 512
#定義一個 MAX_LEN 的變數，表示分詞後的序列最大長度。
encoded_review = tokenizer.encode_plus(
    test_text,
    max_length=MAX_LEN,
    add_special_tokens=True,
    return_token_type_ids=False,
    return_attention_mask=True,
    return_tensors='pt',
    truncation=True
)
"""
使用 BERT 分詞器 tokenizer 的 encode_plus 方法對文本進行分詞和轉換。
test_text 是要處理的文本序列。
max_length=MAX_LEN設置最大序列長度為之前定義的 MAX_LEN。
add_special_tokens=True 添加特殊的分詞符號。
return_token_type_ids=False 不返回 token_type_ids，這在單句文本中不需要。
return_attention_mask=True 返回 attention mask，用於指示模型在處理輸入時應該關注的部分。
return_tensors='pt' 返回 PyTorch 張量格式。
truncation=True 對輸入的序列進行截斷，使得序列的長度不超過最大長度。
"""

In [ ]:
encoded_review
#encoded_review包含已經編碼過的文本序列的字典。

{'input_ids': tensor([[  101,   136,   136,   136,   136,   136,   136,   136,  3680,   819,
          1419,  3300,  8114,  8882,  6737, 11382,   511,  6737, 11382,  1377,
          1347,  5645,  5204,  4495,  5162,   147,  1086,  4495,  8024,  1060,
          5442,  1072,  1295,  1398,   868,  4500,  8024,  1377,   914,  6868,
          3173,  7376,   807,  6342,   511,   924,  6362,  2542,  4472,  7518,
          3268,  5204,  2898,   978,  2434,   511,  3268,  7077,  3189,  3315,
          4638,  3297,   881,  5401,  2159,  5469,  1501,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
test_model = model.to('cuda')
#將模型移動到 GPU 上進行運算。

input_ids = encoded_review['input_ids'].to('cuda')
#將輸入序列的索引表示移動到 GPU 上進行運算。
attention_mask = encoded_review['attention_mask'].to('cuda')
#將注意力遮罩移動到 GPU 上進行運算。
output = test_model(input_ids, attention_mask)
#使用移動到 GPU 上的模型對輸入序列進行預測，並獲得預測結果。
result = torch.argmax(output[0][0])
#從模型的輸出中獲取預測結果。
classnames = ['合格', '不合格']
#定義了類別名稱列表
print(f'廣告詞: {test_text}')
#列印待分類的文本序列。
print(f'檢測結果: {classnames[result]}')
#根據預測結果索引在 classnames 列表中找到對應的類別名稱。

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx